In [1]:
import pandas as pd
import numpy as np

In [2]:
trainData = pd.read_csv('datasets/train.csv')
testData = pd.read_csv('datasets/test.csv')

In [3]:
print("Training data size: {}; Test data size: {}".format(len(trainData), len(testData)))

Training data size: 891; Test data size: 418


In [4]:
trainData.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [5]:
testData.head(3)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q


In [6]:
def mergeData(base, append, target):
    base = base.drop(target, 1)
    combined = base.append(append)
    combined.reset_index(inplace = True)
    combined.drop('index', axis = 1, inplace = True)
    return combined

In [7]:
allData = mergeData(trainData, testData, 'Survived')

## Fill In Age Gaps

Several rows have a missing age. To fill the gaps we will find the median of ages based on other passengers with a similar `sex`, `pclass`, and `honorific`.

This is accomplished in several steps.

1. Extract honorific from name.
2. Create a uniform mapping of all the honorifics.
3. Group the data and capture the median based on `sex`, `pclass`, and the mapped `honorific`.
4. Map any passengers that have a missing age to the data from the median.

In [8]:
allData.Name[0:5]

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
Name: Name, dtype: object

In [9]:
honorifics = set(allData.Name.map(lambda name: name.split(',')[1].split('.')[0].strip()))

In [10]:
allData['Hon'] = allData.Name.map(lambda name: name.split(',')[1].split('.')[0].strip())

In [11]:
honorifics

{'Capt',
 'Col',
 'Don',
 'Dona',
 'Dr',
 'Jonkheer',
 'Lady',
 'Major',
 'Master',
 'Miss',
 'Mlle',
 'Mme',
 'Mr',
 'Mrs',
 'Ms',
 'Rev',
 'Sir',
 'the Countess'}

In [12]:
HONORIFIC_MAP = {
    'Capt': 'Military',
    'Col': 'Military',
    'Don': 'Nobility',
    'Dona': 'Nobility',
    'Dr': 'Doctor',
    'Jonkheer': 'Nobility',
    'Lady': 'Nobility',
    'Major': 'Military',
    'Master': 'Boy',
    'Miss': 'Ms',
    'Mlle': 'Ms',
    'Mme': 'Mrs',
    'Mr': 'Mr',
    'Mrs': 'Mrs',
    'Ms': 'Ms',
    'Sir': 'Mr',
    'the Countess': 'Nobility'
}

In [13]:
allData.Hon = allData.Hon.map(HONORIFIC_MAP)

In [14]:
# You can use pandas.isnull() to see if your mapping missed any honorifics
pd.isnull(allData['Hon']).any()

True

In [15]:
# To see which remaining columns have nulls explicitly,
allData.columns[pd.isnull(allData).any()].tolist()

['Age', 'Fare', 'Cabin', 'Embarked', 'Hon']

In [16]:
# To see which rows corresponding to a particular column explicitly,
col = 'Hon'
allData[allData[col].isnull()]

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Hon
149,150,2,"Byles, Rev. Thomas Roussel Davids",male,42.0,0,0,244310,13.000,NaN,S,NaN
150,151,2,"Bateman, Rev. Robert James",male,51.0,0,0,S.O.P. 1166,12.525,NaN,S,NaN
249,250,2,"Carter, Rev. Ernest Courtenay",male,54.0,1,0,244252,26.000,NaN,S,NaN
626,627,2,"Kirkland, Rev. Charles Leonard",male,57.0,0,0,219533,12.350,NaN,Q,NaN
848,849,2,"Harper, Rev. John",male,28.0,0,1,248727,33.000,NaN,S,NaN
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.000,NaN,S,NaN
1040,1041,2,"Lahtinen, Rev. William",male,30.0,1,1,250651,26.000,NaN,S,NaN
1055,1056,2,"Peruschitz, Rev. Joseph Maria",male,41.0,0,0,237393,13.000,NaN,S,NaN


In [17]:
# Reset allData['Hon'] to the pre-mapped values
allData['Hon'] = allData.Name.map(lambda name: name.split(',')[1].split('.')[0].strip())

# Redefine dictionary so that all Rev. honorifics get mapped
HONORIFIC_MAP = {
    'Capt': 'Military',
    'Col': 'Military',
    'Don': 'Nobility',
    'Dona': 'Nobility',
    'Dr': 'Doctor',
    'Jonkheer': 'Nobility',
    'Lady': 'Nobility',
    'Major': 'Military',
    'Master': 'Boy',
    'Miss': 'Ms',
    'Mlle': 'Ms',
    'Mme': 'Mrs',
    'Mr': 'Mr',
    'Mrs': 'Mrs',
    'Ms': 'Ms',
    'Sir': 'Mr',
    'the Countess': 'Nobility',
    'Rev' : 'Rev'
}

allData.Hon = allData.Hon.map(HONORIFIC_MAP)

In [18]:
# To see which remaining columns have nulls explicitly,
allData.columns[pd.isnull(allData).any()].tolist()

['Age', 'Fare', 'Cabin', 'Embarked']

In [19]:
# No remaining null Honorifics

In [20]:
grouped_results_by_class = allData.groupby(['Sex', 'Pclass', 'Hon']).median()

In [21]:
grouped_results_by_class

PassengerId   Age  SibSp  Parch      Fare
Sex    Pclass Hon                                                
female 1      Doctor          797.0  49.0    0.0    0.0   25.9292
              Mrs             853.5  45.0    1.0    0.0   78.1125
              Ms              529.5  30.0    0.0    0.0   99.9625
              Nobility        760.0  39.0    0.0    0.0   86.5000
       2      Mrs             547.0  30.5    1.0    0.0   26.0000
              Ms              597.0  20.0    0.0    0.0   19.5000
       3      Mrs             658.0  31.0    1.0    1.0   15.5000
              Ms              613.0  18.0    0.0    0.0    8.0500
male   1      Boy             803.0   6.0    1.0    2.0  134.5000
              Doctor          661.0  47.0    1.0    0.0   81.8583
              Military        695.0  53.0    0.0    0.0   30.5000
              Mr              632.5  42.0    0.0    0.0   48.3021
              Nobility        427.0  39.0    0.0    0.0   13.8604
       2      Boy             550.0   2.0    1.0    1.0   26.0000
              Doctor          358.5  38.5    0.0    0.0   12.2500
              Mr              723.5  30.0    0.0    0.0   13.0000
              Rev             738.0  41.5    0.0    0.0   13.0000
       3      Boy             789.0   6.0    3.0    1.0   22.3583
              Mr              640.5  26.0    0.0    0.0    7.8958

In [22]:
def missingAgeFromSimilarMedian(grouped_median, row):
    return grouped_median['Age'][row.Sex][row.Pclass][row.Hon]

allData.Age = allData.apply(lambda row: missingAgeFromSimilarMedian(grouped_results_by_class, row) if np.isnan(row.Age) else row.Age, axis = 1)

In [23]:
allData.head(10)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Hon
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Ms
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,Mr
5,6,3,"Moran, Mr. James",male,26.0,0,0,330877,8.4583,NaN,Q,Mr
6,7,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,Mr
7,8,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,Boy
8,9,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,Mrs
9,10,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,Mrs


In [24]:
# Encode Honorifics into scalar values
allData['Hon'] = pd.get_dummies(allData['Hon'])

In [25]:
# Now that they're scalar value, we can use describe() to check the count
allData.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare,Hon
count,1309.000000,1309.000000,1309.000000,1309.000000,1309.000000,1308.000000,1309.000000
mean,655.000000,2.294882,29.275149,0.498854,0.385027,33.295479,0.046600
std,378.020061,0.837836,13.454072,1.041658,0.865560,51.758668,0.210862
min,1.000000,1.000000,0.170000,0.000000,0.000000,0.000000,0.000000
25%,328.000000,2.000000,21.000000,0.000000,0.000000,7.895800,0.000000
50%,655.000000,3.000000,26.000000,0.000000,0.000000,14.454200,0.000000
75%,982.000000,3.000000,36.500000,1.000000,0.000000,31.275000,0.000000
max,1309.000000,3.000000,80.000000,8.000000,9.000000,512.329200,1.000000


In [26]:
# Now we can see we're missing a single value from the Fare column